In [37]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture("lunge_1.mp4")

# Curl counter variables
counter = 0 
stage = None

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    #print(a,b,c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image = cv2.resize(image,(960,640))
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            left_ankle_visibility=landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            right_ankle_visibility=landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            if (left_ankle_visibility <0.60) and (right_ankle_visibility <0.60) :
                print(right_ankle_visibility)
                
                cv2.putText(image, str("please go back"), 
                                            (200,50), 
                                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2, cv2.LINE_AA)
                        
            else:
                
                # Get coordinates
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    
                # Calculate angle
                left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        
                
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
               
                right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

                # print(right_knee_angle)
                # Visualize angle
                cv2.putText(image, str(left_knee_angle), 
                               tuple(np.multiply(left_knee, [960, 640]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA
                                    )
                cv2.putText(image, str(right_knee_angle), 
                               tuple(np.multiply(right_knee, [960, 640]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA
                                    )
                
                # Curl counter logic
                if left_knee_angle > 170 and right_knee_angle > 170:
                    stage = "up"
                    print(left_knee_angle,right_knee_angle)
                if left_knee_angle < 90 and stage =='up' and right_knee_angle <90 :
                    stage="down"
                    counter +=1
                    print(counter)

        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):q
            break

    cap.release()
    cv2.destroyAllWindows()

172.1077767613967 175.5242561626031
175.96186048455158 172.7031989733156
177.57326804372673 170.83062480518944
178.58978743946662 170.5532140076579
178.1361031627711 171.13589068825058
178.69754124166877 172.33369344034134
179.4938556184848 175.17298440667284
179.94402507061204 177.21345317198308
179.4881551223478 178.48641643296918
179.03779586312484 179.24832778233812
178.6620437544074 179.88031850177134
178.4969764807344 179.729017500154
178.59392241447063 179.8055026729663
178.5452056969081 179.41929325919943
178.43757874500605 179.1683687620882
178.5355581407599 178.9812661891492
178.60044544745475 178.89339935621788
178.63643694443894 178.69669991059095
176.36670336621765 179.94429900595264
179.9440050962169 178.7048057516344
178.61362856235755 178.97771097755708
177.85349635714252 178.70262244432288
178.1759922592223 178.60794512070532
177.6698810857303 178.69608058636283
176.97077229741518 178.5091911382917
176.52576159760943 178.11438451996176
176.82634540476695 178.3968157909

179.97355973765698 177.17935749834209
179.7509151717836 174.7059385266988
178.0099570336671 171.8664302829265
177.95391680078998 170.3398989113618
179.1557132307177 174.2303052059093
178.7485546619433 177.97595959052177
178.99319687188677 178.8064950588433
179.60449279103503 178.36207530739165
179.9715704892596 177.62131891351896
179.38174979957736 176.95170139304855
179.0506098035867 177.20891889426335
179.17132758358812 178.15642694644777
179.4831668977716 178.12493097984387
179.78211734828207 177.78286761278272
179.9751691647762 177.1291338055671
179.68373984411542 176.65655685832488
179.24390573930881 170.84801900937154
172.82590437736584 170.28984950557427
170.04764506303258 172.20649314275494
178.37263891401355 172.94824435132938
179.66925506529162 176.8699656100062
179.71669265869687 179.27167606694712
179.7401477691396 179.77214580647362
179.12066006554866 179.8983774381858
179.21442841648422 179.96686205019859
179.30165325494826 179.0707975153965
179.3687578819119 179.05656096

error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
